# 5. Runtime Analysis

In [5]:
import numpy as np
import time
import pandas as pd
from dtw_algorithm import get_accum_cost_and_steps, get_path

# Conduct runtime experiments

In [2]:
def get_settings(algorithm):

    steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
    weights = np.array([2,3,3])
    warp_max, subsequence = None, False

    if algorithm == 'DTW2':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights = np.array([1,2,2])
    elif algorithm == 'DTW3':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
    elif algorithm == 'DTW4':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,1])
    elif algorithm == 'DTW5':
        steps = np.array([0,1,1,0]).reshape((-1,2))
        weights = np.array([1,1])
    elif algorithm == 'DTW1_add3':
        steps = np.array([1,1,1,2,2,1,1,3,3,1]).reshape((-1,2))
        weights = np.array([2,3,3,4,4])
    elif algorithm == 'DTW1_add4':
        steps = np.array([1,1,1,2,2,1,1,3,3,1,1,4,4,1]).reshape((-1,2))
        weights = np.array([2,3,3,4,4,5,5])

    elif algorithm == 'adaptiveWeight1':
        steps = np.array([0,1,1,0]).reshape((-1,2))
    elif algorithm == 'adaptiveWeight2':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))

    elif algorithm == 'selectiveTransitions2':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, warp_max = np.array([1,1,2]), 2
    elif algorithm == 'selectiveTransitions3':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, warp_max = np.array([1,1,2]), 3
    elif algorithm == 'selectiveTransitions4':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, warp_max = np.array([1,1,2]), 4
    elif algorithm == 'selectiveTransitions5':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, warp_max = np.array([1,1,2]), 5

    elif algorithm == 'SubDTW1':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,2]), True
    elif algorithm == 'SubDTW2':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights, subsequence = np.array([2,3,3]), True
    elif algorithm == 'SubDTW3':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights, subsequence = np.array([1,2,2]), True
    elif algorithm == 'SubDTW4':
        steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,1]), True
    elif algorithm == 'SubDTW5':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, subsequence = np.array([0,1,1]), True
    elif algorithm == 'SubDTW6':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,1]), True
    elif algorithm == 'SubDTW7':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,2]), True

    elif algorithm == 'SubDTW3_add3':
        steps = np.array([1,1,1,2,2,1,1,3,3,1]).reshape((-1,2))
        weights, subsequence = np.array([1,2,2,1,3]), True
    elif algorithm == 'SubDTW6_add3':
        steps = np.array([0,1,1,0,1,1,1,3,3,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,1,1,3]), True
    elif algorithm == 'SubDTW3_add4':
        steps = np.array([1,1,1,2,2,1,1,3,3,1,1,4,4,1]).reshape((-1,2))
        weights, subsequence = np.array([1,2,2,1,3,1,4]), True
    elif algorithm == 'SubDTW6_add4':
        steps = np.array([0,1,1,0,1,1,1,3,3,1,1,4,4,1]).reshape((-1,2))
        weights, subsequence = np.array([1,1,1,1,3,1,4]), True

    elif algorithm == 'SubDTW_selectiveTransitions2':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
        warp_max, subsequence = 2, True
    elif algorithm == 'SubDTW_selectiveTransitions3':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
        warp_max, subsequence = 3, True
    elif algorithm == 'SubDTW_selectiveTransitions4':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
        warp_max, subsequence = 4, True
    elif algorithm == 'SubDTW_selectiveTransitions5':
        steps = np.array([0,1,1,0,1,1]).reshape((-1,2))
        weights = np.array([1,1,2])
        warp_max, subsequence = 5, True

    return steps, weights, warp_max, subsequence

In [6]:
def alignDTW(F1, F2, algorithm):

    res = []
    steps, weights, warp_max, subsequence = get_settings(algorithm)
    N, M = F1.shape[1], F2.shape[1] # we assume that N >= M
    res.append(time.time())
    
    # apply downsampling or adaptive weights
    if algorithm == "downsampleQuantized":
        # we wish to only select M columns of of F1 to get (12 x M)
        index = [int(round(x)) for x in np.linspace(0, N-1, M)]
        F1 = F1[:, index]
    elif algorithm == "downsampleInterpolate":
        # we want to multiply matrix (12 x N) by (N x M) to get (12 x M)
        transform = np.zeros((N, M))
        index = np.linspace(0, N-1, M) # M indices evenly spaced between [0, N-1]
        for col, index in enumerate(index):
            # at column m, insert weight RIGHT at position ROW and LEFT at position ROW+1
            row = int(index)
            right = index - int(index)
            left = 1 - right
            # if we are at the last row, insert weight 1
            if row + 1 == N:
                transform[row, col] = 1
                continue
            transform[row, col] = left
            transform[row+1, col] = right
        F1 = F1 @ transform
    elif algorithm == "upsampleQuantized":
        index = [int(x) for x in np.linspace(0, M-1, N)]
        F2 = F2[:, index] 
    elif algorithm == "upsampleInterpolate":
        # we want to multiply matrix (12 x M) by (M x N) to get (12 x N)
        transform = np.zeros((M, N))
        index = np.linspace(0, M-1, N) # N indices evenly spaced between [0, M-1]
        for col, index in enumerate(index):
            # at column N, insert weight RIGHT at position ROW and LEFT at position ROW+1
            row = int(index)
            right = index - int(index)
            left = 1 - right
            # if we are at the last row, insert weight 1
            if row + 1 == M:
                transform[row, col] = 1
                continue
            transform[row, col] = left
            transform[row+1, col] = right
        F2 = F2 @ transform
    elif algorithm == "adaptiveWeight1":
        weights = np.array([N/M, 1])
    elif algorithm == "adaptiveWeight2":
        weights = np.array([N/M, 1, 1 + N/M])
    res.append(time.time())

    # compute cost matrix
    C = 1 - F1.T @ F2
    res.append(time.time())

    # run DTW algorithm
    x_steps = steps[:,0].astype(np.uint32) # horizontal steps
    y_steps = steps[:,1].astype(np.uint32) # veritcal steps
    params = {'x_steps': x_steps, 'y_steps': y_steps, 'weights': weights, 'subsequence': subsequence}
    D, s = get_accum_cost_and_steps(C, params, warp_max)
    res.append(time.time())

    # retrieve paths and steps taken
    path, _, _, track_steps = get_path(D, s, params)
    res.append(time.time())

    if algorithm == "downsampleQuantized" or algorithm == "downsampleInterpolate":
        path[0] = path[0] * N / M
    elif algorithm == "upsampleQuantized" or algorithm == "upsampleInterpolate":
        path[1] = path[1] * M / N
    res.append(time.time())
    
    return np.diff(res)

In [ ]:
algos = ['DTW1', 'DTW2', 'DTW3', 'DTW4', 'DTW5', 'DTW1_add3', 'DTW1_add4', 'downsampleQuantized', 'downsampleInterpolate', 'upsampleQuantized', 'upsampleInterpolate', 'adaptiveWeight1', 'adaptiveWeight2', 'selectiveTransitions2','selectiveTransitions3','selectiveTransitions4','selectiveTransitions5']
Ns = [100, 300, 1_000, 3_000, 10_000, 30_000]
Ks = [1, 2, 4]

with open('run_time.csv', 'w') as f:
    for algo in algos:
        for N in Ns:
            for K in Ks:
                for _ in range(10):

                    F1 = np.random.rand(12,N)
                    F2 = np.random.rand(12,N*K)
                    times = alignDTW(F1, F2, algo)
                    times = ",".join(map(str, times))
                    f.write(f'{algo},{N},{N*K},{times}\n')

# Perform analysis

In [10]:
df =  pd.read_csv('run_time.csv', header=None)
for i in range(1,df.shape[1]):
    df[i] = df[i].apply(lambda x: float(x))
df.columns = ["algorithm", "N", "M", "adjust_sample_adaptive", "compute_cost_matrix", "get_accum_cost_and_steps", "retrieve_path", "adjust_sample"]
for column in ["adjust_sample_adaptive", "compute_cost_matrix", "get_accum_cost_and_steps", "retrieve_path", "adjust_sample"]:
    df[column] *= 1000

In [13]:
mean = df.groupby(["algorithm", "N", "M"]).mean()
mean

adjust_sample_adaptive  compute_cost_matrix  \
algorithm N       M                                                       
DTW1      100.0   100.0                   0.002980             5.591011   
                  200.0                   0.000954             0.371194   
                  400.0                   0.000882             0.838208   
          300.0   300.0                   0.000858             1.737165   
                  600.0                   0.000834             3.503108   
                  1200.0                  0.000930             4.817581   
          1000.0  1000.0                  0.001025             5.618143   
                  2000.0                  0.001144             9.575057   
                  4000.0                  0.001168            21.718979   
          3000.0  3000.0                  0.001097            43.745184   
                  6000.0                  0.001287            96.766543   
                  12000.0                 0.001311           205.058408   
          10000.0 10000.0                 0.001407           459.064341   
                  20000.0                 0.001764          1096.562934   
                  40000.0                 0.002241          2163.307190   
          30000.0 30000.0                 0.002050          4141.589618   
                  60000.0                 0.003958         10142.623019   
                  120000.0                0.011134         20478.252435   
DTW2      100.0   100.0                   0.001049             0.046659   
                  200.0                   0.001287             0.186038   
                  400.0                   0.000739             0.133300   
          300.0   300.0                   0.000668             0.242281   
                  600.0                   0.000668             0.398922   
                  1200.0                  0.000691             0.720668   
          1000.0  1000.0                  0.001001             2.210879   
                  2000.0                  0.001597             9.001780   
                  4000.0                  0.001669            21.152782   
          3000.0  3000.0                  0.001550            41.646242   
                  6000.0                  0.001717            87.607312   
                  12000.0                 0.001931           172.800946   
          10000.0 10000.0                 0.001550           467.394471   
                  20000.0                 0.001645          1012.824917   
                  40000.0                 0.003600          2094.289446   
          30000.0 30000.0                 0.001979          4073.815417   
                  60000.0                 0.005174          9375.432682   
                  120000.0                0.010800         20558.950377   
DTW3      100.0   100.0                   0.002599             0.103116   
                  200.0                   0.001001             0.124216   
                  400.0                   0.000811             0.115061   
          300.0   300.0                   0.000787             0.207877   
                  600.0                   0.000620             0.356960   
                  1200.0                  0.000668             0.551629   
          1000.0  1000.0                  0.000834             2.292562   
                  2000.0                  0.001335             9.126854   
                  4000.0                  0.001383            21.007776   
          3000.0  3000.0                  0.001454            41.921139   
                  6000.0                  0.001526            90.043521   
                  12000.0                 0.001550           190.467119   
          10000.0 10000.0                 0.001717           427.203131   
                  20000.0                 0.002408           986.637473   
                  40000.0                 0.001907          2109.750414   

                            get_accum_cost_and_steps

In [14]:
std = df.groupby(["algorithm", "N", "M"]).std()
std

adjust_sample_adaptive  compute_cost_matrix  \
algorithm N       M                                                       
DTW1      100.0   100.0                   0.004912            17.346921   
                  200.0                   0.000373             0.719592   
                  400.0                   0.000161             0.220613   
          300.0   300.0                   0.000123             0.481663   
                  600.0                   0.000169             1.039940   
                  1200.0                  0.000285             1.232343   
          1000.0  1000.0                  0.000196             1.440852   
                  2000.0                  0.000188             1.345370   
                  4000.0                  0.000176             3.910234   
          3000.0  3000.0                  0.000230             1.959983   
                  6000.0                  0.000302             1.564479   
                  12000.0                 0.000169             5.246816   
          10000.0 10000.0                 0.000307            16.341069   
                  20000.0                 0.001013           108.975188   
                  40000.0                 0.001222           170.327358   
          30000.0 30000.0                 0.000981           314.368735   
                  60000.0                 0.001830          1366.015644   
                  120000.0                0.006499          1630.177115   
DTW2      100.0   100.0                   0.001232             0.048919   
                  200.0                   0.000302             0.195858   
                  400.0                   0.000307             0.014578   
          300.0   300.0                   0.000151             0.023537   
                  600.0                   0.000188             0.017992   
                  1200.0                  0.000176             0.024746   
          1000.0  1000.0                  0.000101             0.095759   
                  2000.0                  0.000390             1.873271   
                  4000.0                  0.000318             1.285618   
          3000.0  3000.0                  0.000169             1.679280   
                  6000.0                  0.000333             2.603786   
                  12000.0                 0.000285             8.427094   
          10000.0 10000.0                 0.000203            60.663268   
                  20000.0                 0.000496           104.917445   
                  40000.0                 0.004857           146.377842   
          30000.0 30000.0                 0.001084           116.848601   
                  60000.0                 0.004511           201.814136   
                  120000.0                0.007775          1835.470065   
DTW3      100.0   100.0                   0.003791             0.091123   
                  200.0                   0.000474             0.128484   
                  400.0                   0.000123             0.005142   
          300.0   300.0                   0.000115             0.015196   
                  600.0                   0.000123             0.032062   
                  1200.0                  0.000151             0.021462   
          1000.0  1000.0                  0.000303             0.333121   
                  2000.0                  0.000607             1.761734   
                  4000.0                  0.000188             0.996247   
          3000.0  3000.0                  0.000567             1.280019   
                  6000.0                  0.000704             2.830977   
                  12000.0                 0.000303            14.075144   
          10000.0 10000.0                 0.000333            16.775945   
                  20000.0                 0.001284            54.251816   
                  40000.0                 0.000318           125.837093   

                            get_accum_cost_and_steps